#### 01 - Ingest Azure Compute Documentation into databricks Unity Catalog

This notebook downloads Azure Compute documentation from GitHub, cleans Markdown content, and writes the data into a Unity Catalog–managed Delta table.

**Execution environment**
- Run this notebook on **Azure Databricks (Premium tier)** with **Unity Catalog enabled**
- Uses a **single-user Databricks cluster** (DBR 15+)
- Writes data as **Unity Catalog–managed Delta tables**

**What this notebook does**
- Downloads the latest Azure Compute documentation from the public GitHub repository  
  `MicrosoftDocs/azure-compute-docs` using a shallow Git clone
- Parses and cleans Markdown files under the `articles/` directory
- Extracts metadata such as document ID, category, title, source URL, and ingestion time
- Persists the processed documents into a governed Delta table:
databricks_rag_demo.default.raw_azure_compute_docs

This notebook establishes the **raw document ingestion layer** for a
Retrieval-Augmented Generation (RAG) pipeline and intentionally avoids
legacy DBFS-based storage in favor of **Unity Catalog–managed data objects**.


In [0]:
%sql

----- workspace will create a default catelog with same name as the workspace, we will mostly work in this catelog
SHOW CATALOGS;

catalog
databricks_rag_demo
samples
system


In [0]:
%sql SHOW SCHEMAS IN databricks_rag_demo;

databaseName
default
information_schema


In [0]:
##### gitpython is a Python wrapper around the git command. We will use this to do git clone

%pip install gitpython # gitpython is a Python wrapper around the git command.

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from git import Repo
import os
import re
from pathlib import Path
from pyspark.sql import Row
from datetime import datetime
from git import Repo
import shutil
import stat

In [0]:
REPO_URL = "https://github.com/MicrosoftDocs/azure-compute-docs.git"

TARGET_DIR = "/tmp/azure-compute-docs" # it will be created on the driver VM’s local disk.
TARGET_ARTICLE_PATH = f"{TARGET_DIR}/articles"

DEFAULT_CATELOG_NAME = "databricks_rag_demo"
TABLE_NAME="raw_azure_compute_docs"

In [0]:
%sh rm -rf /tmp/azure-compute-docs

In [0]:
def download_azure_compute_docs():
    
    # SHALLOW clone
    Repo.clone_from(
        REPO_URL,
        TARGET_DIR,
        depth=1 # Depth = how much git history you download
    )

download_azure_compute_docs()
os.listdir(f"{TARGET_ARTICLE_PATH}")

['service-fabric',
 'virtual-machines',
 'container-instances',
 'azure-impact-reporting',
 'virtual-machine-scale-sets']

In [0]:
# Function to clean markdown text
def clean_markdown(md_text: str) -> str:
    # Remove code blocks
    #md_text = re.sub(r"```.*?```", "", md_text, flags=re.S)
    # Remove images
    md_text = re.sub(r"!\[.*?\]\(.*?\)", "", md_text)
    # Remove links but keep text
    md_text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", md_text)
    # Remove headings symbols
    md_text = re.sub(r"#+ ", "", md_text)
    return md_text.strip()

In [0]:
def prepare_spark_row_data():
    article_path = Path(TARGET_ARTICLE_PATH)
    rows = []
    min_length = 500  # skip stubs / TOCs

    for md_file in article_path.rglob("*.md"):
        try:
            with open(md_file, "r", encoding="utf-8", errors="ignore") as f:
                raw_md = f.read()

            cleaned = clean_markdown(raw_md)

            if len(cleaned) < min_length:
                continue

            rel_path = str(md_file.relative_to(article_path))
            category = rel_path.split("/", 1)[0] if "/" in rel_path else rel_path

            rows.append(Row(
                doc_id=rel_path,
                source="azure-compute-docs",
                category=category,  # e.g. virtual-machines
                title=md_file.stem,
                raw_text=cleaned,
                url=f"https://learn.microsoft.com/en-us/azure/{md_file.relative_to(TARGET_ARTICLE_PATH)}",
                ingest_time=datetime.utcnow()
            ))

        except Exception as e:
            # Fail-safe: skip bad files
            continue
    return rows

rows = prepare_spark_row_data()
docs_df = spark.createDataFrame(rows)

display(docs_df.limit(2))



/home/spark-c6c441ab-7e09-494b-bf0b-82/.ipykernel/1886/command-8004489376597638-4245322713:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ingest_time=datetime.utcnow()


doc_id source category title raw_text url ingest_time service-fabric/service-fabric-best-practices-infrastructure-as-code.md azure-compute-docs service-fabric service-fabric-best-practices-infrastructure-as-code ---
title: Azure Service Fabric infrastructure as Code Best Practices
description: Best practices and design considerations for managing Azure Service Fabric as infrastructure as code.
ms.topic: concept-article
ms.author: tomcassidy
author: tomvcassidy
ms.service: azure-service-fabric
services: service-fabric
ms.date: 07/14/2022
Customer intent: "As a cloud administrator, I want to utilize Infrastructure as Code to deploy and manage Azure Service Fabric clusters, so that I can ensure consistent and efficient resource configuration and maintenance."
---

Infrastructure as code

In a production scenario, create Azure Service Fabric clusters using Resource Manager templates. Resource Manager templates provide greater control of resource properties and ensure that you have a consistent resource model.

Sample Resource Manager templates are available for Windows and Linux in the Azure samples on GitHub. These templates can be used as a starting point for your cluster template. Download `azuredeploy.json` and `azuredeploy.parameters.json` and edit them to meet your custom requirements.

!INCLUDE [updated-for-az]

To deploy the `azuredeploy.json` and `azuredeploy.parameters.json` templates you downloaded above, use the following Azure CLI commands:

```azurecli
ResourceGroupName="sfclustergroup"
Location="westus"

az group create --name $ResourceGroupName --location $Location 
az deployment group create --name $ResourceGroupName --template-file azuredeploy.json --parameters @azuredeploy.parameters.json
```

Creating a resource using PowerShell

```powershell
$ResourceGroupName="sfclustergroup"
$Location="westus"
$Template="azuredeploy.json"
$Parameters="azuredeploy.parameters.json"

New-AzResourceGroup -Name $ResourceGroupName -Location $Location
New-AzResourceGroupDeployment -Name $ResourceGroupName -TemplateFile $Template -TemplateParameterFile $Parameters
```

Service Fabric resources

You can deploy applications and services onto your Service Fabric cluster via Azure Resource Manager. See Manage applications and services as Azure Resource Manager resources for details. The following are best practice Service Fabric application specific resources to include in your Resource Manager template resources.

```json
{
 "apiVersion": "2019-03-01",
 "type": "Microsoft.ServiceFabric/clusters/applicationTypes",
 "name": "[concat(parameters('clusterName'), '/', parameters('applicationTypeName'))]",
 "location": "[variables('clusterLocation')]",
},
{
 "apiVersion": "2019-03-01",
 "type": "Microsoft.ServiceFabric/clusters/applicationTypes/versions",
 "name": "[concat(parameters('clusterName'), '/', parameters('applicationTypeName'), '/', parameters('applicationTypeVersion'))]",
 "location": "[variables('clusterLocation')]",
},
{
 "apiVersion": "2019-03-01",
 "type": "Microsoft.ServiceFabric/clusters/applications",
 "name": "[concat(parameters('clusterName'), '/', parameters('applicationName'))]",
 "location": "[variables('clusterLocation')]",
},
{
 "apiVersion": "2019-03-01",
 "type": "Microsoft.ServiceFabric/clusters/applications/services",
 "name": "[concat(parameters('clusterName'), '/', parameters('applicationName'), '/', parameters('serviceName'))]",
 "location": "[variables('clusterLocation')]"
}
```

To deploy your application using Azure Resource Manager, you first must create a sfpkg Service Fabric Application package. The following Python script is an example of how to create a sfpkg:

```python
Create SFPKG that needs to be uploaded to Azure Storage Blob Container
microservices_sfpkg = zipfile.ZipFile(
 self.microservices_app_package_name, 'w', zipfile.ZIP_DEFLATED)
package_length = len(self.microservices_app_package_path)

for root, dirs, files in os.walk(self.microservices_app_package_path):
 root_folder = root[package_len

In [0]:
# In Databricks notebooks, spark is: a pre-initialized SparkSession object that Databricks creates for you.

type(spark)

pyspark.sql.connect.session.SparkSession

In [0]:
def save_to_delta_table(docs_df):

    # write to warehouse
    (
        docs_df
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(f"{DEFAULT_CATELOG_NAME}.default.{TABLE_NAME}")
    )

save_to_delta_table(docs_df)

In [0]:
%sql

--- Validate data is stored into the table successfully
SELECT COUNT(*) FROM databricks_rag_demo.default.raw_azure_compute_docs

count(1)
1751


In [0]:
%sql

-- delete this table if needed, this will clean up the environment
DROP TABLE databricks_rag_demo.default.raw_azure_compute_docs;